In [5]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType

# driver = webdriver.Chrome(service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# python
import pandas as pd
import time

In [6]:
# define driver
driver = webdriver.Chrome()

# open web page
url = f"https://www.lemonde.fr/recherche/?search_keywords=ukraine&start_at=19/12/1994&end_at=15/12/2022&search_sort=date_desc&page=1"
driver.get(url)

# accept cookies
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="js-body"]/div[6]/div/footer/button'))).click()
time.sleep(1)

In [7]:
# get the number of pages for the search
driver.get(url)
last_page = int(driver.find_elements(by=By.XPATH, value='/html/body/main/article/section/section[1]/section[2]/section[4]/a[5]')[0].text)
last_page

250

In [8]:
# create list of dict of title and date for each article about ukraine
articles = []

for page in range(1, last_page):
    url = f"https://www.lemonde.fr/recherche/?search_keywords=ukraine&start_at=19/12/1994&end_at=15/12/2022&search_sort=date_desc&page={page}"
    driver.get(url)
    driver_title = driver.find_elements(by=By.XPATH, value='/html/body/main/article/section/section[1]/section[2]/section[3]/section/a/h3')
    driver_date = driver.find_elements(by=By.XPATH, value='/html/body/main/article/section/section[1]/section[2]/section[3]/section/p/span[1]')
    for title, date in zip(driver_title, driver_date):
        articles.append({"title": title.text, "date": date.text})

driver.quit()

In [9]:
# number of articles found
len(articles)

9243

In [10]:
# create dataframe from dict
df = pd.DataFrame.from_dict(articles)

In [11]:
df

,title,date
0,Guerre en Ukraine en direct : les Etats-Unis i...,"Publié le 15 décembre 2022 à 18h25, mis à jour..."
1,« Xi Jinping a signé l’arrêt de mort de l’inte...,"Publié le 09 décembre 2022 à 14h48, mis à jour..."
2,Crise énergétique : « La France est sur le poi...,"Publié le 12 décembre 2022 à 04h00, mis à jour..."
3,"« Pour soutenir l’Ukraine, il faut aider les a...","Publié le 12 décembre 2022 à 17h00, mis à jour..."
4,"« Nous, députés européens, voulons plus d’Euro...",Publié le 13 décembre 2022 à 05h30
...,...,...
9238,Gazprom reprend ses livraisons de gaz à l’Ukraine,"Publié le 12 octobre 2015 à 12h47, mis à jour ..."
9239,"En Biélorussie, un scrutin pour renouer avec l...","Publié le 10 octobre 2015 à 01h39, mis à jour ..."
9240,"Sainte Angela, prix Nobel de la paix ?",Publié le 08 octobre 2015 à 19h14
9241,"Face à la présence russe en Syrie, l’OTAN évoq...","Publié le 08 octobre 2015 à 11h16, mis à jour ..."


In [14]:
# create csv
df.to_csv('../data/le_monde.csv')